# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\torre\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv("NEW_API_KEY")

In [4]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key = api_key)

In [6]:
# Fetch the Bitcoin news articles
btc_headlines = newsapi.get_everything(
    q = "bitcoin", language = "en", sort_by = "relevancy")


In [7]:
# Fetch the Ethereum news articles
eth_headlines = newsapi.get_everything(
    q = "ethereum", language = "en", sort_by = "relevancy")

In [8]:
# Create the Bitcoin sentiment scores DataFrame
sentiments = []

for articles in btc_headlines["articles"]:
    try:
        text = articles["content"]
        results = analyzer.polarity_scores(text)
        compound = results["compound"]
        pos = results["pos"]
        neu = results["neu"]
        neg = results["neg"]

        sentiments.append({
            "text": text,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
        })
    except AttributeError:
        pass
    
btc  = pd.DataFrame(sentiments)
btc.head()

,text,Compound,Positive,Negative,Neutral
0,"Even in cyberspace, the Department of Justice ...",0.7351,0.147,0.000,0.853
1,"""Bitcoin was seen by many of its libertarian-l...",-0.7713,0.000,0.169,0.831
2,Feb 22 (Reuters) - Bitcoin miners are feeling ...,-0.1779,0.046,0.067,0.887
3,March 1 (Reuters) - Bitcoin has leapt since Ru...,0.0000,0.000,0.000,1.000
4,Whose side is cryptocurrency on? If you had as...,-0.3182,0.055,0.091,0.854


In [9]:
# Create the Ethereum sentiment scores DataFrame
sentiments = []

for articles in eth_headlines["articles"]:
    try:
        text = articles["content"]
        results = analyzer.polarity_scores(text)
        compound = results["compound"]
        pos = results["pos"]
        neu = results["neu"]
        neg = results["neg"]

        sentiments.append({
            "text": text,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
        })
    except AttributeError:
        pass
    
eth  = pd.DataFrame(sentiments)
eth.head()

,text,Compound,Positive,Negative,Neutral
0,"In February, shit hit the fan in the usual way...",-0.3182,0.059,0.093,0.848
1,"Technical analysis isnt a perfect tool, but it...",-0.2498,0.000,0.059,0.941
2,People in Colorado will be able to pay their s...,-0.1027,0.000,0.036,0.964
3,Coinbase reported that the share of trading vo...,0.6705,0.188,0.000,0.812
4,Illustration by James Bareham / The Verge\r\n\...,-0.4588,0.000,0.083,0.917


In [10]:
# Describe the Bitcoin Sentiment
btc.describe()

,Compound,Positive,Negative,Neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.018970,0.052900,0.043750,0.903300
std,0.427134,0.049112,0.061198,0.070857
min,-0.771300,0.000000,0.000000,0.739000
25%,-0.231275,0.000000,0.000000,0.853750
50%,0.000000,0.051500,0.000000,0.913000
75%,0.345300,0.078250,0.071000,0.952500
max,0.735100,0.152000,0.187000,1.000000


In [11]:
# Describe the Ethereum Sentiment
eth.describe()

,Compound,Positive,Negative,Neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.133310,0.071000,0.032200,0.896750
std,0.409553,0.084548,0.040875,0.093457
min,-0.526700,0.000000,0.000000,0.692000
25%,-0.110175,0.000000,0.000000,0.841000
50%,0.000000,0.051500,0.000000,0.913000
75%,0.528675,0.120500,0.059000,0.973000
max,0.834000,0.249000,0.115000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereum has the higher mean positve score.

Q: Which coin had the highest compound score?

A: Ethereum has the highest compound score. 

Q. Which coin had the highest positive score?

A: Ethereum has the highest positve score on this day. 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [12]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [13]:
# Instantiate the lemmatizer
wnl = WordNetLemmatizer()

# Create a list of stopwords
stop = stopwords.words("english")

# Expand the default stopwords list if necessary
stop.append("u")
stop.append("it'")
stop.append("'s")
stop.append("n't")
stop.append('…')
stop.append("\`")
stop.append('``')
stop.append('char')
stop.append("''")
stop = set(stop)

In [19]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    words = word_tokenize(text)

    # Remove the punctuation from text
    words = list(
        filter(lambda t: t not in punctuation, words))
   
    # Convert the words to lowercase
    
    words = list(
        filter(lambda t: t.lower(), words))
    
    # Remove the stop words
   
    words = list(
        filter(lambda t: t.lower() not in stop, words))
      
    # Lemmatize words into root words
    tokens = [wnl.lemmatize(word) for word in words]
    
    return tokens

In [20]:
# Create a new tokens column for Bitcoin
btc["tokens"] = btc.text.apply(tokenizer)
btc.head()

,text,Compound,Positive,Negative,Neutral,tokens
0,"Even in cyberspace, the Department of Justice ...",0.7351,0.147,0.000,0.853,"[Even, cyberspace, Department, Justice, able, ..."
1,"""Bitcoin was seen by many of its libertarian-l...",-0.7713,0.000,0.169,0.831,"[Bitcoin, seen, many, libertarian-leaning, fan..."
2,Feb 22 (Reuters) - Bitcoin miners are feeling ...,-0.1779,0.046,0.067,0.887,"[Feb, 22, Reuters, Bitcoin, miner, feeling, he..."
3,March 1 (Reuters) - Bitcoin has leapt since Ru...,0.0000,0.000,0.000,1.000,"[March, 1, Reuters, Bitcoin, leapt, since, Rus..."
4,Whose side is cryptocurrency on? If you had as...,-0.3182,0.055,0.091,0.854,"[Whose, side, cryptocurrency, asked, Satoshi, ..."


In [21]:
# Create a new tokens column for Ethereum
eth["tokens"] = eth.text.apply(tokenizer)
eth.head()

,text,Compound,Positive,Negative,Neutral,tokens
0,"In February, shit hit the fan in the usual way...",-0.3182,0.059,0.093,0.848,"[February, shit, hit, fan, usual, way, old, tw..."
1,"Technical analysis isnt a perfect tool, but it...",-0.2498,0.000,0.059,0.941,"[Technical, analysis, isnt, perfect, tool, may..."
2,People in Colorado will be able to pay their s...,-0.1027,0.000,0.036,0.964,"[People, Colorado, able, pay, state, tax, cryp..."
3,Coinbase reported that the share of trading vo...,0.6705,0.188,0.000,0.812,"[Coinbase, reported, share, trading, volume, e..."
4,Illustration by James Bareham / The Verge\r\n\...,-0.4588,0.000,0.083,0.917,"[Illustration, James, Bareham, Verge, 15, mill..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [22]:
from collections import Counter
from nltk import ngrams

In [23]:
# Generate the Bitcoin N-grams where N=2
N = 2 
grams = ngrams(tokenizer(btc.text.str.cat()), N)
Counter(grams).most_common(20)

[(('Reuters', 'Bitcoin'), 3),
 (('since', 'Russia'), 3),
 (('char', 'Bitcoin'), 2),
 (('char', 'Feb'), 2),
 (('char', 'March'), 2),
 (('March', '15'), 2),
 (('/li', 'li'), 2),
 (('char', 'SAN'), 2),
 (('SAN', 'SALVADOR'), 2),
 (('Reuters', 'El'), 2),
 (('El', 'Salvador'), 2),
 (('Central', 'American'), 2),
 (('15', 'million'), 2),
 (('Even', 'cyberspace'), 1),
 (('cyberspace', 'Department'), 1),
 (('Department', 'Justice'), 1),
 (('Justice', 'able'), 1),
 (('able', 'use'), 1),
 (('use', 'tried'), 1),
 (('tried', 'true'), 1)]

In [24]:
# Generate the Ethereum N-grams where N=2
N = 2 
grams = ngrams(tokenizer(eth.text.str.cat()), N)
Counter(grams).most_common(20)

[(('char', 'cryptocurrency'), 3),
 (('cryptocurrency', 'boom'), 3),
 (('boom', 'past'), 3),
 (('past', 'year'), 3),
 (('year', 'helped'), 3),
 (('helped', 'propel'), 3),
 (('propel', 'newer'), 3),
 (('newer', 'market'), 3),
 (('market', 'record'), 3),
 (('record', 'height'), 3),
 (('height', 'digital'), 3),
 (('digital', 'collectible'), 3),
 (('collectible', 'known'), 3),
 (('known', 'NFTs'), 3),
 (('NFTs', 'fact'), 3),
 (('fact', 'total'), 3),
 (('total', 'NFT'), 3),
 (('NFT', 'sale'), 3),
 (('sale', 'volume'), 3),
 (('volume', 'hit'), 3)]

In [25]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [27]:
# Use token_count to get the top 10 words for Bitcoin
alltokens = tokenizer(btc.text.str.cat())
token_count(alltokens,10)

[('char', 20),
 ('Bitcoin', 10),
 ('Reuters', 7),
 ('Russia', 6),
 ('March', 5),
 ('cryptocurrency', 4),
 ('Ukraine', 4),
 ('bitcoin', 4),
 ('15', 4),
 ('said', 3)]

In [28]:
# Use token_count to get the top 10 words for Ethereum
alltokens = tokenizer(eth.text.str.cat())
token_count(alltokens,10)

[('char', 20),
 ('Ethereum', 5),
 ('cryptocurrency', 5),
 ('year', 5),
 ('volume', 5),
 ('past', 5),
 ('hit', 4),
 ('crypto', 4),
 ('NFTs', 4),
 ('newsletter', 4)]

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---